# T3: Analize de Dados do SiaSUS

Forma: Trabalho em grupo com apresentação do ambiente e do código para o professor.

Objetivo:
Nesta primeira parte, que deve estar pronta para a aula do dia 03/11/2025, os grupos devem obter, preparar e carregar em um banco de dados relacional os dados do SIASUS (Sistema de Informações Ambulatoriais do SUS), disponibilizados publicamente pelo DataSUS.

Instruções:
1. 1) Obtenção da base de dados
 - Acesse o repositório do DataSUS e faça o download dos arquivos do SIASUS no formato .dbc.
 - Selecione os arquivos referentes a procedimentos ambulatoriais (ex.: PARS e outros complementares necessários para a descrição da tabela PARS).
2. 2) Conversão de formato
 - Converta os arquivos .dbc para .dbf utilizando as ferramentas recomendadas (ex.: TABWIN).
 - Certifique-se de validar a integridade dos arquivos após a conversão.
3) Visualização dos dados
 - Utilize o TABWIN, software oficial do DataSUS, para visualizar os arquivos .dbf.
 - A partir do TABWIN, gere:
  - Scripts SQL de criação de tabelas.
4) Carga dos dados em SGBD:
 - Insira os dados no banco de dados relacional de sua escolha (ex. MySQL, PostgreSQL, IBM DB2, etc).
 - Para arquivos menores, utilize diretamente os scripts de inserção gerados pelo TABWIN.
 - Para arquivos muito grandes (como o PARS, que contém milhões de registros), gere um arquivo CSV a partir do .dbf e utilize um comando de LOAD (carga em lote) no SGBD escolhido.
 - Se optar pelo MySQL (que foi o utilizado em aula):
  - Pesquise e utilize a instrução LOAD DATA na linha de comando do servidor (para evitar problemas de permissões com o cliente Workbench) conforme a documentação oficial: https://dev.mysql.com/doc/refman/8.4/en/load-data.html
Exemplo de comando LOAD DATA no terminal do servidor: LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/PARS2508.csv' INTO TABLE pars CHARACTER SET latin1 FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES  TERMINATED BY '\r\n' IGNORE 1 ROWS; 
5) A continuidade do trabalho, na aula do dia 03/11/2025, depende do ambiente do banco de dados funcionando.

## Principais libs que usaremos: 

In [11]:
# Esse projeto foi feito com o objetivo educativo para a matéria de Programção para Ciência dos dados
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt



In [12]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# /kaggle/input/sus-data-csv/CADGERRS.csv

/kaggle/input/TB_SIGTAW.csv
/kaggle/input/PARS2501.csv
/kaggle/input/tb_municip.csv
/kaggle/input/S_CID.csv
/kaggle/input/PARS2508.csv
/kaggle/input/CBO.csv
/kaggle/input/CADGERRS.csv
/kaggle/input/rl_municip_micibge.csv
/kaggle/input/PARS2505.csv
/kaggle/input/tb_micibge.csv


# Declarando arquivos:

In [13]:
import pandas as pd

# Define o caminho base para facilitar (opcional, mas limpo)
base_path = '/kaggle/input/'

# --- 1. TAREFAS FATO (Os eventos, a produção) ---
# Estas são as tabelas principais. Elas registram os eventos (atendimentos).
# Os nomes (PARS2501, PARS2505, PARS2508) sugerem que são dados mensais
# (ex: Janeiro, Maio, Agosto de 2025). Vamos carregá-los e juntá-los.

print("Carregando tabelas de produção (PARS)...")

# O que é: Tabela FATO - Produção Ambulatorial (Parte 1, ex: Mês 1)
# O que vamos contar: Registros de procedimentos, valores, local, paciente, etc.
df_prod_01 = pd.read_csv(base_path + 'PARS2501.csv', encoding='latin1', low_memory=False)

# O que é: Tabela FATO - Produção Ambulatorial (Parte 2, ex: Mês 5)
# O que vamos contar: Mais registros de procedimentos...
df_prod_05 = pd.read_csv(base_path + 'PARS2505.csv', encoding='latin1', low_memory=False)

# O que é: Tabela FATO - Produção Ambulatorial (Parte 3, ex: Mês 8)
# O que vamos contar: Mais registros de procedimentos...
df_prod_08 = pd.read_csv(base_path + 'PARS2508.csv', encoding='latin1', low_memory=False)

# --- 2. TAREFAS DIMENSÃO (Os dicionários, "de-para") ---
# Estas são as tabelas auxiliares que dão nome aos códigos das tabelas FATO.

print("Carregando tabelas de dimensão (dicionários)...")

# O que é: Tabela de Procedimentos (SIGTAP)
# O que vamos usar: Dar nome aos códigos de procedimento (ex: '0301010072' -> 'CONSULTA MEDICA')
df_procedimentos = pd.read_csv(base_path + 'TB_SIGTAW.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Municípios (IBGE)
# O que vamos usar: Dar nome aos códigos de município (ex: '431020' -> 'Ijuí').
# ESSENCIAL para a análise de fluxo de pacientes (item 4 do seu trabalho).
df_municipios = pd.read_csv(base_path + 'tb_municip.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Doenças (Classificação Internacional de Doenças - CID)
# O que vamos usar: Dar nome aos códigos de diagnóstico (ex: 'I10' -> 'Hipertensão Essencial').
df_cid = pd.read_csv(base_path + 'S_CID.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Ocupações (Classificação Brasileira de Ocupações - CBO)
# O que vamos usar: Dar nome aos códigos de ocupação do profissional
# (ex: '225125' -> 'Médico Clínico').
df_cbo = pd.read_csv(base_path + 'CBO.csv', encoding='latin1', low_memory=False)

# O que é: Cadastro de Estabelecimentos (CNES)
# O que vamos usar: Dar nome e endereço aos códigos de hospitais, clínicas e UBS
# (ex: '2254611' -> 'HOSPITAL DE CARIDADE DE IJUI').
df_estabelecimentos = pd.read_csv(base_path + 'CADGERRS.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Relação (RL) Município -> Microrregião
# O que vamos usar: Ligar um código de município a um código de microrregião.
df_rl_mun_micro = pd.read_csv(base_path + 'rl_municip_micibge.csv', encoding='latin1', low_memory=False)

# O que é: Tabela de Microrregiões (IBGE)
# O que vamos usar: Dar nome ao código da microrregião (ex: '11001' -> 'Porto Velho').
df_microrregioes = pd.read_csv(base_path + 'tb_micibge.csv', encoding='latin1', low_memory=False)

print("\nTodos os arquivos foram carregados com sucesso!")

Carregando tabelas de produção (PARS)...
Carregando tabelas de dimensão (dicionários)...

Todos os arquivos foram carregados com sucesso!


In [14]:
# Junta as 3 tabelas de produção (uma em cima da outra)
df_producao_total = pd.concat([df_prod_01, df_prod_05, df_prod_08], ignore_index=True)

# Libera a memória, já que não precisamos mais das partes separadas
del df_prod_01
del df_prod_05
del df_prod_08

print(f"Tabelas de produção combinadas!")
print(f"Total de registros para analisar: {len(df_producao_total)}")



Tabelas de produção combinadas!
Total de registros para analisar: 13210711


In [15]:
df_producao_total.head(n=10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,R,0,NaN,0.0,0.0,0.0,121003.0,NaN,3999
1,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,R,0,NaN,0.0,0.0,0.0,121003.0,NaN,3999
2,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
3,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
4,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
5,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
6,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
7,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
8,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
9,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999


In [16]:
print(df_producao_total.columns.tolist())

['PA_CODUNI', 'PA_GESTAO', 'PA_CONDIC', 'PA_UFMUN', 'PA_REGCT', 'PA_INCOUT', 'PA_INCURG', 'PA_TPUPS', 'PA_TIPPRE', 'PA_MN_IND', 'PA_CNPJCPF', 'PA_CNPJMNT', 'PA_CNPJ_CC', 'PA_MVM', 'PA_CMP', 'PA_PROC_ID', 'PA_TPFIN', 'PA_SUBFIN', 'PA_NIVCPL', 'PA_DOCORIG', 'PA_AUTORIZ', 'PA_CNSMED', 'PA_CBOCOD', 'PA_MOTSAI', 'PA_OBITO', 'PA_ENCERR', 'PA_PERMAN', 'PA_ALTA', 'PA_TRANSF', 'PA_CIDPRI', 'PA_CIDSEC', 'PA_CIDCAS', 'PA_CATEND', 'PA_IDADE', 'IDADEMIN', 'IDADEMAX', 'PA_FLIDADE', 'PA_SEXO', 'PA_RACACOR', 'PA_MUNPCN', 'PA_QTDPRO', 'PA_QTDAPR', 'PA_VALPRO', 'PA_VALAPR', 'PA_UFDIF', 'PA_MNDIF', 'PA_DIF_VAL', 'NU_VPA_TOT', 'NU_PA_TOT', 'PA_INDICA', 'PA_CODOCO', 'PA_FLQT', 'PA_FLER', 'PA_ETNIA', 'PA_VL_CF', 'PA_VL_CL', 'PA_VL_INC', 'PA_SRV_C', 'PA_INE', 'PA_NAT_JUR']


In [17]:
# A chave da nossa tabela principal (produção) é 'PA_MUNPCN'
chave_principal = 'PA_MUNPCN'

# A chave da nossa tabela auxiliar (municípios) é 'CO_MUNICIP'
chave_auxiliar = 'CO_MUNICIP'

# Converte ambas as colunas-chave para string (texto) para garantir o merge
df_producao_total[chave_principal] = df_producao_total[chave_principal].astype(str)
df_municipios[chave_auxiliar] = df_municipios[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_MUNPCN' e 'CO_MUNICIP' corrigidos para string!


In [18]:
# Vamos criar nosso dataframe de análise final, começando com este merge
# Puxamos apenas as colunas de nome e UF da tabela de municípios
df_analise = pd.merge(
    left=df_producao_total,
    right=df_municipios[['CO_MUNICIP', 'DS_NOME', 'CO_UF']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '431020')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '431020')
    how='left'                 # 'left' garante que não vamos perder nenhuma linha da principal
)

print("Merge com Municípios concluído!")

# Vamos verificar o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")

# Vamos renomear as colunas para ficar mais claro
df_analise.rename(columns={'DS_NOME': 'MUNICIPIO_PACIENTE', 'CO_UF': 'UF_PACIENTE'}, inplace=True)

# Mostra as colunas originais e as novas que foram "puxadas"
print(df_analise[['PA_MUNPCN', 'MUNICIPIO_PACIENTE', 'UF_PACIENTE']].head())

Merge com Municípios concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_MUNPCN MUNICIPIO_PACIENTE UF_PACIENTE
0    431720         Santa Rosa          43
1    430960        Horizontina          43
2    999999           Invalido          XX
3    999999           Invalido          XX
4    999999           Invalido          XX


In [19]:
# A chave da nossa tabela de análise é 'PA_PROC_ID'
chave_principal = 'PA_PROC_ID'

# A chave da nossa tabela auxiliar (procedimentos) é 'IP_COD'
chave_auxiliar = 'IP_COD'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_procedimentos[chave_auxiliar] = df_procedimentos[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_PROC_ID' e 'IP_COD' corrigidos para string!


In [20]:
# Agora fazemos o merge no 'df_analise' (que já tem os nomes dos municípios)
# Vamos puxar apenas a coluna 'IP_DSCR' (Descrição do Procedimento)
df_analise = pd.merge(
    left=df_analise,
    right=df_procedimentos[['IP_COD', 'IP_DSCR']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '0301010072')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '0301010072')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Procedimentos concluído!")

# Vamos renomear a nova coluna para ficar claro
df_analise.rename(columns={'IP_DSCR': 'NOME_PROCEDIMENTO'}, inplace=True)

# Verifica o resultado, mostrando a coluna do código e a nova coluna com o nome
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_PROC_ID', 'NOME_PROCEDIMENTO']].head())

Merge com Procedimentos concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_PROC_ID                         NOME_PROCEDIMENTO
0  206010079      TOMOGRAFIA COMPUTADORIZADA DO CRANIO
1  206020031       TOMOGRAFIA COMPUTADORIZADA DE TORAX
2  301010072  CONSULTA MEDICA EM ATENCAO ESPECIALIZADA
3  301010072  CONSULTA MEDICA EM ATENCAO ESPECIALIZADA
4  301010072  CONSULTA MEDICA EM ATENCAO ESPECIALIZADA


In [21]:
# A chave da nossa tabela de análise é 'PA_CODUNI'
chave_principal = 'PA_CODUNI'

# A chave da nossa tabela auxiliar (estabelecimentos) é 'CNES'
chave_auxiliar = 'CNES'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_estabelecimentos[chave_auxiliar] = df_estabelecimentos[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_CODUNI' e 'CNES' corrigidos para string!


In [22]:
# Vamos puxar o 'NOME FANTASIA' e a 'RAZAO SOCIAL' do estabelecimento
df_analise = pd.merge(
    left=df_analise,
    right=df_estabelecimentos[['CNES', 'FANTASIA', 'RAZ_SOCI']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '2254611')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '2254611')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Estabelecimentos concluído!")

# Renomeia as colunas para ficar mais claro
df_analise.rename(columns={
    'FANTASIA': 'NOME_ESTABELECIMENTO',
    'RAZ_SOCI': 'RAZAO_SOCIAL_ESTAB'
}, inplace=True)

# Verifica o resultado, mostrando o código e os novos nomes
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CODUNI', 'NOME_ESTABELECIMENTO']].head())

Merge com Estabelecimentos concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_CODUNI NOME_ESTABELECIMENTO
0   2254611  HOSPITAL VIDA SAUDE
1   2254611  HOSPITAL VIDA SAUDE
2   2254611  HOSPITAL VIDA SAUDE
3   2254611  HOSPITAL VIDA SAUDE
4   2254611  HOSPITAL VIDA SAUDE


In [23]:
# A chave da nossa tabela de análise é 'PA_CIDPRI'
chave_principal = 'PA_CIDPRI'

# A chave da nossa tabela auxiliar (CID) é 'CD_COD'
chave_auxiliar = 'CD_COD'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_cid[chave_auxiliar] = df_cid[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_CIDPRI' e 'CD_COD' corrigidos para string!


In [24]:
# Vamos puxar a 'CD_DESCR' (Descrição do CID)
df_analise = pd.merge(
    left=df_analise,
    right=df_cid[['CD_COD', 'CD_DESCR']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: 'I10')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: 'I10')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Diagnósticos (CID) concluído!")

# Renomeia a coluna para ficar mais claro
df_analise.rename(columns={'CD_DESCR': 'NOME_DIAGNOSTICO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CIDPRI', 'NOME_DIAGNOSTICO']].head())

Merge com Diagnósticos (CID) concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_CIDPRI   NOME_DIAGNOSTICO
0      R529       R52.9 Dor NE
1      R529       R52.9 Dor NE
2      0000  CID NAO INFORMADO
3      0000  CID NAO INFORMADO
4      0000  CID NAO INFORMADO


In [25]:
# A chave da nossa tabela de análise é 'PA_CBOCOD'
chave_principal = 'PA_CBOCOD'

# A chave da nossa tabela auxiliar (CBO) é 'CBO'
chave_auxiliar = 'CBO'

# Converte ambas as colunas-chave para string (texto)
df_analise[chave_principal] = df_analise[chave_principal].astype(str)
df_cbo[chave_auxiliar] = df_cbo[chave_auxiliar].astype(str)

print(f"Tipos das chaves '{chave_principal}' e '{chave_auxiliar}' corrigidos para string!")

Tipos das chaves 'PA_CBOCOD' e 'CBO' corrigidos para string!


In [26]:
# Vamos puxar a 'DS_CBO' (Descrição do CBO)
df_analise = pd.merge(
    left=df_analise,
    right=df_cbo[['CBO', 'DS_CBO']], 
    left_on=chave_principal,    # Chave da tabela principal (ex: '225125')
    right_on=chave_auxiliar,   # Chave da tabela auxiliar (ex: '225125')
    how='left'                 # 'left' para não perder nenhum registro
)

print("Merge com Ocupações (CBO) concluído!")

# Renomeia a coluna para ficar mais claro
df_analise.rename(columns={'DS_CBO': 'NOME_OCUPACAO'}, inplace=True)

# Verifica o resultado
print("\nVerificando as 5 primeiras linhas do resultado:")
print(df_analise[['PA_CBOCOD', 'NOME_OCUPACAO']].head())

Merge com Ocupações (CBO) concluído!

Verificando as 5 primeiras linhas do resultado:
  PA_CBOCOD                                  NOME_OCUPACAO
0    225320  Medico em radiologia e diagnostico por imagem
1    225320  Medico em radiologia e diagnostico por imagem
2    225203                    Medico em cirurgia vascular
3    225203                    Medico em cirurgia vascular
4    225203                    Medico em cirurgia vascular


# Análise em ijuí

# Merge de tabelas apenas com os dados de "Ijuí"
Neste ponto reduzimos **13.2 milhões** de linhas para **160 mil registros** (os que realmente importam para Ijuí) essa ação é crucial para o bom andamento dos testes a seguir, melhorando muito nossa eficiência!

- O uso de memória caiu de **7.4+ GB** para apenas **86.7 MB**.
- Agora temos uma tabela leve, rápida e com todas as informações que precisamos para a análise

In [27]:
import pandas as pd

print("--- Iniciando Análise Focada em Ijuí ---")

# --- PASSO 1: Encontrar o Código de Ijuí ---
# (Precisamos que a chave df_municipios['CO_MUNICIP'] já seja string)
try:
    ijui_info = df_municipios[df_municipios['DS_NOME'] == 'Ijui']
    codigo_ijui_str = ijui_info['CO_MUNICIP'].values[0]
    
    # A coluna PA_UFMUN (Município do Estabelecimento) é int64 (número)
    # Precisamos converter o código de Ijuí para int para o filtro funcionar
    codigo_ijui_int = int(codigo_ijui_str)
    
    print(f"Código de Ijuí encontrado: {codigo_ijui_int} (como número)")

except Exception as e:
    print(f"ERRO: Não foi possível encontrar o código de Ijuí. Verifique se df_municipios está carregado.")
    print(f"Detalhe do erro: {e}")

# --- PASSO 2: Filtrar a Tabela Gigante (13M de linhas) ---
print(f"Filtrando {len(df_producao_total)} registros para focar apenas em Ijuí...")

# Criamos um NOVO DataFrame SÓ com atendimentos feitos em estabelecimentos de Ijuí
# Usamos .copy() para garantir que é um objeto novo e evitar erros de memória
df_ijui_producao = df_producao_total[df_producao_total['PA_UFMUN'] == codigo_ijui_int].copy()

print(f"Filtro Concluído! Tabela reduzida para {len(df_ijui_producao)} registros.")

# --- PASSO 3: Preparar Chaves da Nova Tabela de Ijuí ---
# Agora, convertemos as chaves de merge *desta tabela pequena* para string
print("Preparando chaves da tabela de Ijuí para os merges...")
df_ijui_producao['PA_MUNPCN'] = df_ijui_producao['PA_MUNPCN'].astype(str)
df_ijui_producao['PA_PROC_ID'] = df_ijui_producao['PA_PROC_ID'].astype(str)
df_ijui_producao['PA_CODUNI'] = df_ijui_producao['PA_CODUNI'].astype(str)
df_ijui_producao['PA_CIDPRI'] = df_ijui_producao['PA_CIDPRI'].astype(str)
df_ijui_producao['PA_CBOCOD'] = df_ijui_producao['PA_CBOCOD'].astype(str)

# --- PASSO 4: Executar Merges na Tabela Pequena de Ijuí ---
print("Iniciando merges...")

# Merge 1: Município do Paciente (Onde o paciente mora)
df_analise = pd.merge(df_ijui_producao, df_municipios[['CO_MUNICIP', 'DS_NOME', 'CO_UF']], 
                      left_on='PA_MUNPCN', right_on='CO_MUNICIP', how='left')
df_analise.rename(columns={'DS_NOME': 'MUNICIPIO_PACIENTE', 'CO_UF': 'UF_PACIENTE'}, inplace=True)
print("Merge 1/5 (Município Paciente) OK")

# Merge 2: Nome do Procedimento
df_analise = pd.merge(df_analise, df_procedimentos[['IP_COD', 'IP_DSCR']], 
                      left_on='PA_PROC_ID', right_on='IP_COD', how='left')
df_analise.rename(columns={'IP_DSCR': 'NOME_PROCEDIMENTO'}, inplace=True)
print("Merge 2/5 (Nome Procedimento) OK")

# Merge 3: Nome do Estabelecimento (Hospital, UBS)
df_analise = pd.merge(df_analise, df_estabelecimentos[['CNES', 'FANTASIA']], 
                      left_on='PA_CODUNI', right_on='CNES', how='left')
df_analise.rename(columns={'FANTASIA': 'NOME_ESTABELECIMENTO'}, inplace=True)
print("Merge 3/5 (Nome Estabelecimento) OK")

# Merge 4: Nome do Diagnóstico (Doença)
df_analise = pd.merge(df_analise, df_cid[['CD_COD', 'CD_DESCR']], 
                      left_on='PA_CIDPRI', right_on='CD_COD', how='left')
df_analise.rename(columns={'CD_DESCR': 'NOME_DIAGNOSTICO'}, inplace=True)
print("Merge 4/5 (Nome Diagnóstico) OK")

# Merge 5: Nome da Ocupação (Profissional)
df_analise = pd.merge(df_analise, df_cbo[['CBO', 'DS_CBO']], 
                      left_on='PA_CBOCOD', right_on='CBO', how='left')
df_analise.rename(columns={'DS_CBO': 'NOME_OCUPACAO'}, inplace=True)
print("Merge 5/5 (Nome Ocupação) OK")

print("\n--- 💾 ANÁLISE DE IJUÍ PRONTA E CARREGADA ---")

# --- PASSO 5: Verificar Resultado ---
# Vamos ver o resultado e o novo uso de memória
df_analise.info()

--- Iniciando Análise Focada em Ijuí ---
Código de Ijuí encontrado: 431020 (como número)
Filtrando 13210711 registros para focar apenas em Ijuí...
Filtro Concluído! Tabela reduzida para 160060 registros.
Preparando chaves da tabela de Ijuí para os merges...
Iniciando merges...
Merge 1/5 (Município Paciente) OK
Merge 2/5 (Nome Procedimento) OK
Merge 3/5 (Nome Estabelecimento) OK
Merge 4/5 (Nome Diagnóstico) OK
Merge 5/5 (Nome Ocupação) OK

--- 💾 ANÁLISE DE IJUÍ PRONTA E CARREGADA ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160060 entries, 0 to 160059
Data columns (total 71 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   PA_CODUNI             160060 non-null  object 
 1   PA_GESTAO             160060 non-null  int64  
 2   PA_CONDIC             160060 non-null  object 
 3   PA_UFMUN              160060 non-null  int64  
 4   PA_REGCT              160060 non-null  int64  
 5   PA_INCOUT             160060 non

In [28]:
# Lista de colunas para remover (quase vazias ou inúteis)
colunas_para_remover = [
    'PA_ETNIA',          # Quase vazia (310 registros)
    'PA_INE',            # Totalmente vazia (0 registros)
    'PA_SRV_C'           # Mais da metade vazia
]

# Apaga as colunas
df_analise.drop(columns=colunas_para_remover, inplace=True)

print("Colunas 'PA_ETNIA', 'PA_INE', e 'PA_SRV_C' foram removidas.")

Colunas 'PA_ETNIA', 'PA_INE', e 'PA_SRV_C' foram removidas.


In [29]:
# Mostra os 15 municípios de pacientes mais frequentes
print("Top 15 municípios de residência dos pacientes:")
print(df_analise['MUNICIPIO_PACIENTE'].value_counts().head(15))

Top 15 municípios de residência dos pacientes:
MUNICIPIO_PACIENTE
Ijui                    78559
Invalido                10439
Panambi                  7529
Joia                     5485
Catuipe                  4524
Augusto Pestana          3710
Crissiumal               3140
Cruz Alta                2856
Santo Augusto            2677
Palmeira Das Missoes     2594
Condor                   2545
Tres Passos              2354
Ajuricaba                2306
Santo Angelo             2035
Bozano                   1427
Name: count, dtype: int64


In [30]:
# Cria um novo DataFrame 'df_analise_limpa'
# que contém APENAS as linhas onde MUNICIPIO_PACIENTE NÃO é 'Invalido'
df_analise_limpa = df_analise[df_analise['MUNICIPIO_PACIENTE'] != 'Invalido'].copy()

print(f"Registros inválidos ('Invalido') removidos.")
print(f"Total de registros original: {len(df_analise)}")
print(f"Total de registros limpos para análise: {len(df_analise_limpa)}")

Registros inválidos ('Invalido') removidos.
Total de registros original: 160060
Total de registros limpos para análise: 149621


In [31]:
# 1. Conta o total de atendimentos limpos
total_atendimentos = len(df_analise_limpa)

# 2. Conta quantos atendimentos são de pacientes que moram em 'Ijui'
atendimentos_de_ijui = len(df_analise_limpa[df_analise_limpa['MUNICIPIO_PACIENTE'] == 'Ijui'])

# 3. Calcula os de fora
atendimentos_de_fora = total_atendimentos - atendimentos_de_ijui

# 4. Calcula as porcentagens
pct_ijui = (atendimentos_de_ijui / total_atendimentos) * 100
pct_fora = (atendimentos_de_fora / total_atendimentos) * 100

# 5. Mostra o resultado para o gestor
print("--- 🏥 Análise de Fluxo Regional de Pacientes (Ijuí) ---")
print(f"Total de atendimentos analisados: {total_atendimentos}")
print(f"\nAtendimentos de pacientes de Ijuí: {atendimentos_de_ijui} ({pct_ijui:.2f}%)")
print(f"Atendimentos de pacientes de OUTROS MUNICÍPIOS: {atendimentos_de_fora} ({pct_fora:.2f}%)")

--- 🏥 Análise de Fluxo Regional de Pacientes (Ijuí) ---
Total de atendimentos analisados: 149621

Atendimentos de pacientes de Ijuí: 78559 (52.51%)
Atendimentos de pacientes de OUTROS MUNICÍPIOS: 71062 (47.49%)


# Top 10 municípios que mais enviam pacientes para Ijuí

In [32]:
# Filtrar o DataFrame para excluir 'Ijui'
df_pacientes_de_fora = df_analise_limpa[df_analise_limpa['MUNICIPIO_PACIENTE'] != 'Ijui']

# Contar os municípios restantes e pegar o Top 10
top_10_municipios = df_pacientes_de_fora['MUNICIPIO_PACIENTE'].value_counts().head(10)

print("--- 🗺️ Top 10 Municípios de Origem dos Pacientes (Excluindo Ijuí) ---")
print(top_10_municipios)

# Preparar dados para o gráfico
df_top_10_chart = top_10_municipios.reset_index()
df_top_10_chart.columns = ['Município', 'Total de Atendimentos']

import altair as alt

# Criar o gráfico de barras horizontal
chart = alt.Chart(df_top_10_chart).mark_bar().encode(
    # Ordenar pela contagem de atendimentos em ordem decrescente
    x=alt.X('Total de Atendimentos:Q'),
    y=alt.Y('Município:N', sort='-x'),
    tooltip=['Município', 'Total de Atendimentos']
).properties(
    title='Top 10 Municípios de Origem dos Pacientes Atendidos em Ijuí'
)

# Salvar o gráfico
chart.save('top_10_municipios_pacientes.json')

--- 🗺️ Top 10 Municípios de Origem dos Pacientes (Excluindo Ijuí) ---
MUNICIPIO_PACIENTE
Panambi                 7529
Joia                    5485
Catuipe                 4524
Augusto Pestana         3710
Crissiumal              3140
Cruz Alta               2856
Santo Augusto           2677
Palmeira Das Missoes    2594
Condor                  2545
Tres Passos             2354
Name: count, dtype: int64


# Quais hospitais, UBS e clínicas mais produzem em Ijuí?

In [33]:
# --- Etapa 1: Limpar os dados "Inválidos" ---
# (df_analise foi recarregado pela célula anterior)
df_analise_limpa = df_analise[df_analise['MUNICIPIO_PACIENTE'] != 'Invalido'].copy()
print(f"Total de registros limpos para análise: {len(df_analise_limpa)}")

# --- Etapa 2: Executar a Análise de Produção por Estabelecimento ---
# Conta a frequência de cada estabelecimento na tabela limpa
ranking_estabelecimentos = df_analise_limpa['NOME_ESTABELECIMENTO'].value_counts().head(10)

print("\n--- 🏥 Ranking de Produção por Estabelecimento (Top 10) ---")
print(ranking_estabelecimentos)

# --- Etapa 3: Gerar Gráfico ---
import altair as alt

# Preparar dados para o gráfico
df_chart_estab = ranking_estabelecimentos.reset_index()
df_chart_estab.columns = ['Estabelecimento', 'Nº de Procedimentos']

# Criar o gráfico de barras horizontal
chart = alt.Chart(df_chart_estab).mark_bar().encode(
    x=alt.X('Nº de Procedimentos:Q'),
    y=alt.Y('Estabelecimento:N', sort='-x'), # '-x' ordena do maior para o menor
    tooltip=['Estabelecimento', 'Nº de Procedimentos']
).properties(
    title='Top 10 Estabelecimentos por Nº de Procedimentos em Ijuí'
)

# Salvar o gráfico
chart.save('top_10_estabelecimentos.json')
print("Gráfico 'top_10_estabelecimentos.json' salvo.")

Total de registros limpos para análise: 149621

--- 🏥 Ranking de Produção por Estabelecimento (Top 10) ---
NOME_ESTABELECIMENTO
HOSPITAL DE CLINICAS IJUI                           58551
HOSPITAL BOM PASTOR IJUI                            26927
17 CRS IJUI                                         14181
CENTRO DE ATENCAO PSICOSSOCIAL CAPS AD              10522
CER III UNIR                                        10017
CENTRO DE ATENCAO PSICOSSOCIAL CAPS INFANTIL         6803
CENTRO AUDITIVO PROAUDI                              5504
POSTO CENTRAL SECRETARIA MUNICIPAL DA SAUDE IJUI     4335
APAE IJUI                                            3903
CENTRO DE ATENCAO PSICOSSOCIAL CAPS II               3751
Name: count, dtype: int64
Gráfico 'top_10_estabelecimentos.json' salvo.


# Quais pessoas mais vão a estes hospitais?